<a href="https://colab.research.google.com/github/Skalwalker/AntiMoneyLaundering/blob/main/anti_money_laundering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
egqrgrewwgerwreggrwegrwergwger

NameError: ignored

In [5]:
dfgdgdfg

NameError: ignored

In [6]:
gGGG

NameError: ignored